In [1]:
import os
import sys
sys.path.append('..')
sys.path.append('../Automatic-Circuit-Discovery/')
sys.path.append('../tracr/')
import IPython
ipython = get_ipython()
if ipython is not None:
    ipython.magic('load_ext autoreload')
    ipython.magic('autoreload 2')
import torch as t
from torch import Tensor

from acdc.docstring.utils import get_all_docstring_things
device = t.device("cuda" if t.cuda.is_available() else "CPU")
print(device)

/tmp/ipykernel_1884/1528838084.py:9: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic('load_ext autoreload')
/tmp/ipykernel_1884/1528838084.py:10: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic('autoreload 2')


cuda


In [2]:
all_docstring_items = get_all_docstring_things(num_examples=40, seq_len=5, device=device, metric_name='docstring_metric', correct_incorrect_wandb=False)

tl_model = all_docstring_items.tl_model
validation_metric = all_docstring_items.validation_metric
validation_data = all_docstring_items.validation_data
validation_labels = all_docstring_items.validation_labels
validation_patch_data = all_docstring_items.validation_patch_data
test_metrics = all_docstring_items.test_metrics
test_data = all_docstring_items.test_data
test_labels = all_docstring_items.test_labels
test_patch_data = all_docstring_items.test_patch_data

Loaded pretrained model attn-only-4l into HookedTransformer
Moving model to device:  cuda


In [3]:
def neg_abs_docstring_metric(logits):
    return -abs(test_metrics['docstring_metric'](logits))

In [4]:
from ACDCPPExperiment import ACDCPPExperiment
import numpy as np
THRESHOLDS = [0.243567]#np.logspace(-4, 1, num=20, base=5) # [0.0473844]
# I'm just using one threshold so I can move fast!

tl_model.reset_hooks()
RUN_NAME = 'acdcpp_best'
acdcpp_exp = ACDCPPExperiment(
    model=tl_model,
    clean_data=test_data,
    corr_data=test_patch_data,
    acdc_metric=neg_abs_docstring_metric,
    acdcpp_metric=test_metrics['docstring_metric'],
    thresholds=THRESHOLDS,
    run_name=RUN_NAME,
    verbose=False,
    attr_absolute_val=True,
    save_graphs_after=0,
    run_acdcpp=True,
    run_acdc=False,
    pruning_mode='edge',
    no_pruned_nodes_attr=1,
)

pruned_heads, num_passes, acdcpp_pruned_attrs, acdc_pruned_attrs, edges_after_acdcpp, edges_after_acdc = acdcpp_exp.run()

  0%|          | 0/1 [00:00<?, ?it/s]WARNING:root:cache_all is deprecated and will eventually be removed, use add_caching_hooks or run_with_cache


self.current_node=TLACDCInterpNode(blocks.3.hook_resid_post, [:])



Edge pruning: 100%|██████████| 226/226 [00:00<00:00, 25294.54it/s]

Edge pruning: 100%|██████████| 226/226 [00:00<00:00, 962.71it/s]

100%|██████████| 1/1 [00:00<00:00,  1.40it/s]


In [7]:
acdcpp_pruned_attrs

{0.0473844: {'blocks.3.hook_resid_post[:]blocks.3.attn.hook_result[:, :, 7]': -0.2686312794685364,
  'blocks.3.hook_resid_post[:]blocks.1.attn.hook_result[:, :, 2]': 0.00038782390765845776,
  'blocks.3.hook_resid_post[:]blocks.0.hook_resid_pre[:]': 0.0,
  'blocks.3.hook_resid_post[:]blocks.3.attn.hook_result[:, :, 6]': -1.3909823894500732,
  'blocks.3.hook_resid_post[:]blocks.1.attn.hook_result[:, :, 1]': 0.0006507146754302084,
  'blocks.3.hook_resid_post[:]blocks.3.attn.hook_result[:, :, 2]': 0.02979740872979164,
  'blocks.3.hook_resid_post[:]blocks.3.attn.hook_result[:, :, 3]': 0.1296483278274536,
  'blocks.3.hook_resid_post[:]blocks.0.attn.hook_result[:, :, 1]': -4.59105140180327e-05,
  'blocks.3.hook_resid_post[:]blocks.2.attn.hook_result[:, :, 5]': 0.01853604055941105,
  'blocks.3.hook_resid_post[:]blocks.2.attn.hook_result[:, :, 6]': 0.04186607897281647,
  'blocks.3.hook_resid_post[:]blocks.0.attn.hook_result[:, :, 0]': 4.978184006176889e-05,
  'blocks.3.hook_resid_post[:]blocks.

In [8]:
import numpy as np
np.quantile(list(acdcpp_pruned_attrs[0.5].values()), [0, 0.25, 0.5, 0.75, 0.9, 1])

array([0.00000000e+00, 1.14791351e-03, 4.88377968e-03, 1.94272548e-02,
       7.03021973e-02, 4.55413342e+00])

In [9]:
import json
with open(f'{RUN_NAME}_acdcpp_scores.json', 'w') as f:
    json.dump(acdcpp_pruned_attrs, f)

In [ ]:
import json
with open(f'{RUN_NAME}_acdc_scores.json', 'w') as f:
    json.dump(acdc_pruned_attrs, f)
with open(f'{RUN_NAME}_num_passes.json', 'w') as f:
    json.dump(num_passes, f)

In [6]:
import json

cleaned_pp_attrs = {}
for thresh in acdcpp_pruned_attrs.keys():
    cleaned_pp_attrs[thresh] = {}
    for (parent, child), attr in acdcpp_pruned_attrs[thresh].items():
        cleaned_pp_attrs[thresh]\
        [f'{child.hook_point_name}{child.replace_parens(child.index)}{parent.hook_point_name}{parent.replace_parens(parent.index)}'] = attr

with open(f'{RUN_NAME}_acdc_pruned_attrs.json', 'w') as f:
    json.dump(cleaned_pp_attrs, f)

In [5]:
import json

for thresh in pruned_heads.keys():
    pruned_heads[thresh][0] = list(pruned_heads[thresh][0])
    pruned_heads[thresh][1] = list(pruned_heads[thresh][1])
    
cleaned_pp_attrs = {}
for thresh in acdcpp_pruned_attrs.keys():
    cleaned_pp_attrs[thresh] = {}
    for (parent, child), attr in acdcpp_pruned_attrs[thresh].items():
        cleaned_pp_attrs[thresh]\
        [f'{child.hook_point_name}{child.replace_parens(child.index)}{parent.hook_point_name}{parent.replace_parens(parent.index)}'] = attr
        
# Cleaning the edges
for thresh in edges_after_acdcpp.keys():
    edges_after_acdcpp[thresh] = list(edges_after_acdcpp[thresh])
    edges_after_acdc[thresh] = list(edges_after_acdc[thresh])
    
APPEND = False

if APPEND:
    with open(f'{RUN_NAME}_pruned_heads.json', 'r') as f:
        old_pruned_heads = json.load(f)
    with open(f'{RUN_NAME}_acdcpp_pruned_attrs.json', 'r') as f:
        old_acdcpp_pruned_attrs = json.load(f)
    with open(f'{RUN_NAME}_acdc_pruned_attrs.json', 'r') as f:
        old_acdc_pruned_attrs = json.load(f)
    with open(f'{RUN_NAME}_num_passes.json', 'r') as f:
        old_num_passes = json.load(f)
    with open(f'{RUN_NAME}_edges_after_acdcpp.json', 'r') as f:
        old_edges_after_acdcpp = json.load(f)
    with open(f'{RUN_NAME}_edges_after_acdc.json', 'r') as f:
        old_edges_after_acdc = json.load(f)
    
    pruned_heads = pruned_heads.update(old_pruned_heads)
    cleaned_pp_attrs = cleaned_pp_attrs.update(old_acdcpp_pruned_attrs)
    acdc_pruned_attrs = acdc_pruned_attrs.update(old_acdc_pruned_attrs)
    num_passes = num_passes.update(old_num_passes)
    edges_after_acdcpp = edges_after_acdcpp.update(old_edges_after_acdcpp)
    edges_after_acdc = edges_after_acdc.update(old_edges_after_acdc)

with open(f'{RUN_NAME}_pruned_heads.json', 'w') as f:
    json.dump(pruned_heads, f)
with open(f'{RUN_NAME}_num_passes.json', 'w') as f:
    json.dump(num_passes, f)
with open(f'{RUN_NAME}_acdcpp_pruned_attrs.json', 'w') as f:
    json.dump(cleaned_pp_attrs, f)
with open(f'{RUN_NAME}_acdc_pruned_attrs.json', 'w') as f:
    json.dump(acdc_pruned_attrs, f)

with open(f'{RUN_NAME}_edges_after_acdcpp.json', 'w') as f:
    json.dump(edges_after_acdcpp, f)
with open(f'{RUN_NAME}_edges_after_acdc.json', 'w') as f:
    json.dump(edges_after_acdc, f)

KeyError: 0.00032